In [1]:
import torch
from torchvision import datasets, transforms
from WongBasedTraining import WongBasedTrainingCIFAR10
from Architectures import PreActResNet18
from Ensemble import Ensemble
from AdversarialAttacks import attack_pgd

In [2]:
train_config = {
    "num_samples_wl": 30000,
    "num_wl": 6,
    "train_eps_wl": 8,
    "training_valrestarts": 1,
    "dataset_name": "cifar10",
    "train_alpha_wl": 10,
    "lr_min_wl": 0,
    "lr_max_wl": 0.2,
    "weight_decay_wl": 5e-4,
    "early_stop_wl": True,
    "delta_init_wl": "random",
    "momentum_wl": 0.9,
    "seed_wl": None,
    "opt_level_wl": "O2",
    "loss_scale_wl": 1.0,
    "delta_init_wl": "random",
    "momentum_wl": 0.9,
    "attack_iters_val_wl": 20,
    "batch_size_wl": 128,
    "restarts_wl": 1,
    "training_method": "wong",
    "adv_train_wl": True,
    "prediction_weights_wl": False,
    "lr_schedule_wl": "cyclic",
    "attack_eps_wl": [0.127],
    "model_base": PreActResNet18,
    "val_attacks": [attack_pgd],
    "dataset": datasets.CIFAR10
}
weakLearners = []
weakLearnerWeights = []

In [3]:
from utils import applyDSTrans
train_ds, test_ds = applyDSTrans(train_config)
train_ds.targets = torch.tensor(np.array(train_ds.targets))
test_ds.targets = torch.tensor(np.array(test_ds.targets))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)
path_head = f"./models/{config['training_method']}/{config['dataset_name']}/{config['num_samples_wl']}Eps{config['train_eps_wl']}/baseline/"
if os.path.exists(path_head):
    print("Already exists, exiting")
else:
    os.mkdir(path_head)
model_base = PreActResNet18

(0.4914, 0.4822, 0.4465) (0.2471, 0.2435, 0.2616)
Normalized DS


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
for t in range(train_config['num_wl']):
    h_i = WongBasedTraining(model_base=model_base, attack_eps=train_config['attack_eps_wl'])
    h_i.fit(train_loader, test_loader)
    model_path = f'{path_head}wl_{t}.pth'
    torch.save(h_i.model.state_dict(), model_path)
    weakLearners.append(model_path)
    weakLearnerWeights.append(1.)

In [ ]:
weight_path = f"{path_head}wl_weights.csv"
with open(weight_path, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(weakLearnerWeights)

In [ ]:
test_config = {
    "num_samples_wl": 30000,
    "num_samples_train": 200,
    "num_samples_val": 1500,
    "train_eps_wl": 8,
    "num_wl": 15,
    "batch_size": 128,
    "testing_attack_iters": 20,
    "testing_restarts": 10,
    "dataset_name": "cifar10",
    "training_method": "wong",
    "train_batch_size": 128,
    "test_batch_size": 128,
    "attack_eps_ensemble": [0.127],
    "auto_attack": True,
    "model_base": PreActResNet18,
    "dataset": datasets.CIFAR10,
    'weak_learner_type': WongBasedTrainingCIFAR10,
    'path': path_head
}


In [ ]:
test_config['results_path'] = f"results/plots/{test_config['training_method']}/{test_config['dataset_name']}/train_eps_{test_config['train_eps_wl']}"

In [ ]:
test_ensemble(test_config)